# Exact diagonalization study of 1/2 Heisenberg chain by Lanczos algorithm

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
# set parameters
N=13
dimension=2**N
#spin chain with all spin-down
z='0'*N
# initialize hamiltonian
H=np.zeros((dimension,dimension))
z

'0000000000000'

In [ ]:
# Matrix Construction
for a in range(dimension):
    state_chain=bin(a)[2:] # the first two should be omitted for this 'bin' function
    l=len(state_chain)
    state_chain=z[0:N-l]+state_chain # make the length equal to N
    #print(state_chain)
  # for PBC, we set i in range(N)
  # for OBC, we set i in range(N-1)
    for i in range(N):
        j=np.mod(i+1,N)
        if state_chain[i]==state_chain[j]: # i=j only diagonal elements
            H[a,a]+=0.25
        else: # else, the raising/lowering operators also have contributions
            H[a,a]-=0.25
            # then exchange i,j
            element_i=state_chain[i]
            element_j=state_chain[j]
            #flip
            if j==0:
#here we are doing the concatenation of string (you can try other methods)
#                    print(state_chain)
                state_chain1=element_i+state_chain[1:N-1]+element_j
            else:
                state_chain1=state_chain[0:i]+element_j+element_i+state_chain[j+1:]
            b=int(state_chain1,2)
            H[a,b]+=0.5

# Lanczos algorithm
https://en.wikipedia.org/wiki/Lanczos_algorithm

In [ ]:
def random_orthogonal(V,j):
    N = V.shape[0]
    v = np.random.randn(N)+1j*np.random.randn(N)
    for i in range(j):
        u = V[:,i]
        v -= np.vdot(u,v)*u/np.vdot(u,u)
    v = v/np.linalg.norm(v)
    return v
def Lanczos(Hamiltonian):
    H = np.copy(Hamiltonian)
    N = H.shape[0]
    m = min(100,N)
    H = 0.5*(H + H.conj().T)  # ensure Hermitian
    V_matrix = np.zeros((N, m), dtype=complex)
    T = np.zeros((m, m), dtype=complex)
    # init
    v = np.random.randn(N) + 1j*np.random.randn(N)
    v /= np.linalg.norm(v)
    V_matrix[:, 0] = v

    w_prime = H @ v
    alpha = np.vdot(v, w_prime)
    w = w_prime - alpha * v
    T[0, 0] = alpha

    for i in range(1, m):
        if (i+1)%(m//10) == 0:
            percent = int((i+1)/m*100)
            print(f"Lanczos Progress:{percent}%")
        h = V_matrix[:, :i].conj().T @ w
        w -= V_matrix[:, :i] @ h
        beta = np.linalg.norm(w)
        v_old = v
        if beta < 10e-6:
            v = random_orthogonal(V_matrix,i)
        else:
            v = w / beta
        V_matrix[:, i] = v
        w_prime = H @ v - beta * v_old
        alpha = np.vdot(v, w_prime)
        w = w_prime - alpha * v

        T[i, i] = alpha
        T[i, i-1] = beta
        T[i-1, i] = beta

    return T




# Implicitly Shifted QR Decomposition Method to solve the eigenvalues of tridiagonal matrix

Algorithm 4 from the book: [Handbook of Linear Algebra by Hogben](https://math.ecnu.edu.cn/~jypan/Teaching/MC/refs/2014%20Symmetric%20Matrix%20Eigenvalue%20Techniques.pdf)




In [ ]:
def Givens(x,y):
    G = np.zeros([2,2])
    if y == 0:
        c = 1
        s = 0
    elif x == 0:
        c = 0
        s = np.sign(y)
    else:
        c = abs(x)/math.sqrt(abs(x)**2+abs(y)**2)
        s = np.sign(x)*y/math.sqrt(abs(x)**2+abs(y)**2)
    G[0,0] = c
    G[1,1] = c
    G[0,1] = s
    G[1,0] = -s
    return G

def QR_iteration(T_matrix):
    T = np.copy(T_matrix)
    T = np.real(T)
    epsilon = 10e-6
    if T.shape == (1,1):
        return T
    n_itr = 0

    while True:
        n_itr += 1
        N = T.shape[0]

        tau = (T[N-2,N-2]-T[N-1,N-1])/2
        sgn = np.copysign(1.0,tau)
        den = tau + sgn * np.sqrt(tau*tau+T[N-1,N-2]**2)
        if den == 0:
            mu = T[N-1,N-1]
        else:
            mu = T[N-1,N-1]-T[N-1,N-2]**2/den

        for i in range(0,N-1):
            if i==0:
                G = Givens(T[0,0]-mu,T[1,0])
            else:
                G = Givens(T[i,i-1],T[i+1,i-1])

            iL = max(i-1,0)
            iR = min(N,i+3)

            T[i:i+2,iL:iR] = G@T[i:i+2,iL:iR]
            T[iL:iR,i:i+2] = T[iL:iR,i:i+2]@G.T

            if abs(T[i,i+1])**2 <= epsilon**2*abs(T[i,i]*T[i+1,i+1]):
                #print(n_itr)
                T[i,i+1] = T[i+1,i] = 0.0
                T[0:i+1,0:i+1] = QR_iteration(T[0:i+1,0:i+1])
                T[i+1:N,i+1:N] = QR_iteration(T[i+1:N,i+1:N])
                return T



In [ ]:
T = Lanczos(H)
Lambda = QR_iteration(T)
eigs_lan = np.diag(Lambda)
eigs_lan = np.sort(eigs_lan)
print(eigs_lan)

Lanczos Progress:10%
Lanczos Progress:20%
Lanczos Progress:30%
Lanczos Progress:40%
Lanczos Progress:50%
Lanczos Progress:60%
Lanczos Progress:70%
Lanczos Progress:80%
Lanczos Progress:90%
Lanczos Progress:100%
[-5.62958433e+00 -5.62958433e+00 -5.00187552e+00 -5.00187541e+00
 -4.92049447e+00 -4.92009354e+00 -4.65676458e+00 -4.41833956e+00
 -4.40146825e+00 -4.35673429e+00 -4.24574167e+00 -4.23715427e+00
 -4.19383491e+00 -4.06506862e+00 -4.06146673e+00 -3.93997452e+00
 -3.87645853e+00 -3.84613605e+00 -3.77611000e+00 -3.75047027e+00
 -3.71165361e+00 -3.62340849e+00 -3.56786298e+00 -3.45201148e+00
 -3.40296342e+00 -3.31732323e+00 -3.26006904e+00 -3.13346937e+00
 -3.04856393e+00 -2.95326103e+00 -2.86587077e+00 -2.77829182e+00
 -2.67311349e+00 -2.58448230e+00 -2.47687280e+00 -2.37299368e+00
 -2.26776412e+00 -2.14389614e+00 -2.06298583e+00 -1.94021539e+00
 -1.83820128e+00 -1.71494176e+00 -1.60846944e+00 -1.49540055e+00
 -1.37715137e+00 -1.26277172e+00 -1.14243882e+00 -1.03403316e+00
 -8.99608

In [ ]:
eig_value_H=np.real(np.linalg.eig(H)[0])
eig_value_H = np.sort(eig_value_H)
print(eig_value_H)

[-5.62958433 -5.62958433 -5.62958433 ...  3.25        3.25
  3.25      ]
